In [9]:
# Set up Chrome WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [2]:
# Install dependencies (run once if not already installed)
# !pip install selenium webdriver-manager pandas beautifulsoup4

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from bs4 import BeautifulSoup
import pandas as pd
import time

# Set up Chrome options (non-headless for visibility)
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

# Initialize the WebDriver
driver = webdriver.Chrome(options=options)

# Store all scraped data
all_data = []

try:
    # Navigate to the page
    print("Navigating to https://www.dba.dk/soeg/?soeg=cykel")
    driver.get("https://www.dba.dk/soeg/?soeg=cykel")

    # Step 1: Handle the Cookie Pop-Up (appears ~1 second after page load)
    print("Attempting to handle cookie consent pop-up...")
    try:
        # Wait for the cookie pop-up iframe to appear (try specific ID first)
        try:
            cookie_iframe = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.ID, "sp_message_iframe_1274320"))
            )
        except TimeoutException:
            # Fallback: Look for any iframe with "sp_message_iframe" in the ID
            print("Specific cookie iframe ID not found, trying fallback...")
            cookie_iframe = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "iframe[id*='sp_message_iframe']"))
            )
        print("Cookie pop-up iframe found.")

        # Switch to the iframe
        driver.switch_to.frame(cookie_iframe)
        print("Switched to cookie pop-up iframe.")

        # Wait for the "Tillad alle" button to be clickable
        accept_all_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button.sp_choice_type_ACCEPT_ALL"))
        )
        accept_all_button.click()
        print("Clicked 'Tillad alle' on cookie pop-up.")

        # Switch back to the default content
        driver.switch_to.default_content()
        print("Switched back to default content.")

        # Wait for the cookie pop-up to disappear
        WebDriverWait(driver, 5).until(
            EC.invisibility_of_element_located((By.CSS_SELECTOR, "div[id*='sp_message_container']"))
        )
        print("Cookie pop-up dismissed.")
    except TimeoutException:
        print("Cookie pop-up not found within 5 seconds. Proceeding...")
        print("Page source for debugging:", driver.page_source[:500])  # Log page source for debugging
        driver.switch_to.default_content()  # Ensure we switch back even if it fails
    except Exception as e:
        print(f"Error handling cookie pop-up: {e}")
        driver.switch_to.default_content()  # Ensure we switch back on error

    # Step 2: Handle the Welcome Pop-Up by Accessing the Shadow DOM
    print("Attempting to handle welcome pop-up by accessing Shadow DOM of <onboarding-modal>...")
    max_attempts = 5
    attempt = 1
    welcome_dismissed = False

    while attempt <= max_attempts and not welcome_dismissed:
        print(f"Attempt {attempt} to find and click 'Påmind mig senere' button in Shadow DOM...")
        try:
            # Wait for the <onboarding-modal> element to be present
            onboarding_modal = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.TAG_NAME, "onboarding-modal"))
            )
            print("Found <onboarding-modal> element.")

            # Use JavaScript to access the Shadow DOM and find the "Påmind mig senere" button
            remind_later_button = driver.execute_script("""
                // Get the <onboarding-modal> element
                const modal = document.querySelector('onboarding-modal');
                if (!modal) return null;

                // Access the Shadow DOM
                const shadowRoot = modal.shadowRoot;
                if (!shadowRoot) return null;

                // Find the "Påmind mig senere" button within the Shadow DOM
                return shadowRoot.querySelector('w-button[variant="secondary"]');
            """)

            if remind_later_button:
                print("Found 'Påmind mig senere' button in Shadow DOM.")
                # Click the button using JavaScript
                driver.execute_script("arguments[0].click();", remind_later_button)
                print("Clicked 'Påmind mig senere' button.")

                # Wait for the <onboarding-modal> element to disappear from the main DOM
                WebDriverWait(driver, 5).until(
                    EC.invisibility_of_element_located((By.TAG_NAME, "onboarding-modal"))
                )
                print("Welcome pop-up dismissed.")
                welcome_dismissed = True
            else:
                print("No 'Påmind mig senere' button found in Shadow DOM on this attempt.")
                attempt += 1
                time.sleep(5)  # Wait 5 seconds before retrying

        except TimeoutException:
            print("No <onboarding-modal> element found on this attempt.")
            attempt += 1
            time.sleep(5)
        except Exception as e:
            print(f"Error handling welcome pop-up on attempt {attempt}: {e}")
            attempt += 1
            time.sleep(5)

    if not welcome_dismissed:
        print("Welcome pop-up not found or dismissed after all attempts. Proceeding...")
        print("Page source for debugging:", driver.page_source[:500])

    # Step 3: Apply Filters (will be reapplied on each page)
    def apply_filters():
        print("Applying filters...")
        max_attempts = 5
        attempt = 1
        filters_applied = False

        while attempt <= max_attempts and not filters_applied:
            print(f"Attempt {attempt} to apply filters...")
            try:
                # Wait for the filters section to be present
                WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "section[aria-labelledby='filters-heading']"))
                )
                print("Filters section found.")

                # Filter 1: Location - København og omegn
                print("Selecting 'København og omegn' filter...")
                copenhagen_checkbox = driver.execute_script("""
                    const label = Array.from(document.querySelectorAll('label')).find(el => 
                        el.textContent.includes('København og omegn')
                    );
                    if (!label) return null;
                    return label.previousElementSibling;
                """)
                if copenhagen_checkbox:
                    print("Found 'København og omegn' checkbox.")
                    driver.execute_script("arguments[0].click();", copenhagen_checkbox)
                    print("Clicked 'København og omegn' filter.")
                else:
                    print("No 'København og omegn' checkbox found on this attempt.")
                    attempt += 1
                    time.sleep(5)
                    continue

                # Filter 2: Ad type - Til salg
                print("Selecting 'Til salg' ad type filter...")
                for_sale_checkbox = driver.execute_script("""
                    const label = Array.from(document.querySelectorAll('label')).find(el => 
                        el.textContent.includes('Til salg')
                    );
                    if (!label) return null;
                    return label.previousElementSibling;
                """)
                if for_sale_checkbox:
                    print("Found 'Til salg' checkbox.")
                    driver.execute_script("arguments[0].click();", for_sale_checkbox)
                    print("Clicked 'Til salg' ad type filter.")
                else:
                    print("No 'Til salg' checkbox found on this attempt.")
                    attempt += 1
                    time.sleep(5)
                    continue

                # Filter 3: Price - From 1500 onwards
                print("Setting price filter from 1500 onwards...")
                price_from_input = driver.execute_script("""
                    const input = document.querySelector('input[name="price_from"]');
                    return input;
                """)
                if price_from_input:
                    print("Found price 'from' input.")
                    driver.execute_script("arguments[0].value = '';", price_from_input)
                    driver.execute_script("arguments[0].value = '1500';", price_from_input)
                    driver.execute_script("arguments[0].dispatchEvent(new Event('input', { bubbles: true }));", price_from_input)
                    print("Set price 'from' to 1500.")
                else:
                    print("No price 'from' input found on this attempt.")
                    attempt += 1
                    time.sleep(5)
                    continue

                price_to_input = driver.execute_script("""
                    const input = document.querySelector('input[name="price_to"]');
                    return input;
                """)
                if price_to_input:
                    print("Found price 'to' input.")
                    driver.execute_script("arguments[0].value = '';", price_to_input)
                    driver.execute_script("arguments[0].dispatchEvent(new Event('input', { bubbles: true }));", price_to_input)
                    print("Cleared price 'to' field for any number onwards.")
                else:
                    print("No price 'to' input found on this attempt.")
                    attempt += 1
                    time.sleep(5)
                    continue

                # Add a small delay to ensure the inputs are processed
                time.sleep(1)

                search_button = driver.execute_script("""
                    const button = Array.from(document.querySelectorAll('button')).find(el => 
                        el.textContent.includes('Søg')
                    );
                    return button;
                """)
                if search_button:
                    print("Found 'Søg' button for price filter.")
                    driver.execute_script("arguments[0].click();", search_button)
                    print("Clicked 'Søg' to apply price filter.")
                else:
                    print("No 'Søg' button found on this attempt.")
                    attempt += 1
                    time.sleep(5)
                    continue

                # Filter 4: Private/Retailer - Privat
                print("Selecting 'Privat' seller filter...")
                private_checkbox = driver.execute_script("""
                    const label = Array.from(document.querySelectorAll('label')).find(el => 
                        el.textContent.includes('Privat')
                    );
                    if (!label) return null;
                    return label.previousElementSibling;
                """)
                if private_checkbox:
                    print("Found 'Privat' checkbox.")
                    driver.execute_script("arguments[0].click();", private_checkbox)
                    print("Clicked 'Privat' seller filter.")
                else:
                    print("No 'Privat' checkbox found on this attempt.")
                    attempt += 1
                    time.sleep(5)
                    continue

                # Wait for the page to fully render after applying filters
                print("Waiting for page to render after applying filters...")
                time.sleep(5)
                filters_applied = True

            except TimeoutException:
                print("Filters section not found on this attempt.")
                attempt += 1
                time.sleep(5)
            except Exception as e:
                print(f"Error applying filters on attempt {attempt}: {e}")
                attempt += 1
                time.sleep(5)

        if not filters_applied:
            print("Filters not applied after all attempts. Proceeding...")

    # Apply filters initially
    apply_filters()

    # Proceed with scraping
    print("Pop-ups handled successfully. Proceeding with scraping...")

    # Loop through 15 pages
    for page in range(1, 15):
        print(f"Scraping page {page}...")

        # Reapply filters on each page to ensure consistency
        apply_filters()

        try:
            WebDriverWait(driver, 60).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "article.sf-search-ad"))
            )
        except Exception as e:
            print(f"Error waiting for listings on page {page}: {e}")
            break

        print("Parsing page source with BeautifulSoup...")
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        # Extract all listing containers using BeautifulSoup
        listings = soup.select("article.sf-search-ad")
        print(f"Found {len(listings)} listings on page {page}")

        if not listings:
            print("No listings found. Stopping.")
            break

        # Scrape each listing using BeautifulSoup
        for idx, listing in enumerate(listings, 1):
            try:
                # Title: h2 a
                title_element = listing.select_one("h2 a")
                title = title_element.get_text(strip=True) if title_element else "N/A"
                print(f"Listing {idx} on page {page} - Title: {title}")

                # Price: div.flex.justify-between.text-m.space-x-12.font-bold.whitespace-nowrap span
                price_element = listing.select_one("div.flex.justify-between.text-m.space-x-12.font-bold.whitespace-nowrap span")
                price = price_element.get_text(strip=True).replace('\xa0', ' ') if price_element else "N/A"
                print(f"Listing {idx} on page {page} - Price: {price}")

                # Location: div.text-xs.s-text-subtle.flex.justify-between.flex-wrap span:first-of-type
                location_element = listing.select_one("div.text-xs.s-text-subtle.flex.justify-between.flex-wrap span:first-of-type")
                location = location_element.get_text(strip=True) if location_element else "N/A"
                print(f"Listing {idx} on page {page} - Location: {location}")

                # Time Posted: div.text-xs.s-text-subtle.flex.justify-between.flex-wrap span:nth-of-type(2)
                time_element = listing.select_one("div.text-xs.s-text-subtle.flex.justify-between.flex-wrap span:nth-of-type(2)")
                time_posted = time_element.get_text(strip=True) if time_element else "N/A"
                print(f"Listing {idx} on page {page} - Time Posted: {time_posted}")

                # Seller Type: div.text-xs.s-text-subtle.truncate span
                seller_type_element = listing.select_one("div.text-xs.s-text-subtle.truncate span")
                seller_type = seller_type_element.get_text(strip=True) if seller_type_element else "N/A"
                print(f"Listing {idx} on page {page} - Seller Type: {seller_type}")

                # Append to data list
                all_data.append({
                    "Title": title,
                    "Price": price,
                    "Location": location,
                    "Time Posted": time_posted,
                    "Seller Type": seller_type
                })
            except Exception as e:
                print(f"Error parsing listing {idx} on page {page} with BeautifulSoup: {e}")
                continue

        # Navigate to the next page (if not on the last page)
        if page < 14:
            try:
                # Wait for the "Next" button to be clickable
                next_button = WebDriverWait(driver, 60).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "a[rel='next']"))
                )
                # Scroll to the button to ensure it's in view
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                # Click the button
                next_button.click()
                print(f"Navigated to page {page + 1}")
                time.sleep(5)
            except Exception as e:
                print(f"Error navigating to page {page + 1} with click: {e}")
                try:
                    # Fallback: Extract the href and navigate directly
                    next_button = driver.find_element(By.CSS_SELECTOR, "a[rel='next']")
                    next_url = next_button.get_attribute("href")
                    driver.get(next_url)
                    print(f"Navigated to page {page + 1} using direct URL: {next_url}")
                    time.sleep(5)
                except Exception as e2:
                    print(f"Error navigating to page {page + 1} with direct URL: {e2}")
                    break

    # Print total listings and first 5 entries
    print(f"Total listings scraped: {len(all_data)}")
    print("First 5 entries:")
    print(all_data[:5])

    # Save to CSV
    df = pd.DataFrame(all_data)
    df.to_csv("dba_bike_listings.csv", index=False)
    print(f"Scraping completed. Saved {len(all_data)} listings to dba_bike_listings.csv")

    # Preview the data
    try:
        df = pd.read_csv("dba_bike_listings.csv")
        print("Preview of the data:")
        print(df.head())
    except Exception as e:
        print(f"Error reading CSV file: {e}")

finally:
    # Close the browser
    print("Closing the browser...")
    driver.quit()

Navigating to https://www.dba.dk/soeg/?soeg=cykel
Attempting to handle cookie consent pop-up...
Specific cookie iframe ID not found, trying fallback...
Cookie pop-up iframe found.
Switched to cookie pop-up iframe.
Clicked 'Tillad alle' on cookie pop-up.
Switched back to default content.
Cookie pop-up dismissed.
Attempting to handle welcome pop-up by accessing Shadow DOM of <onboarding-modal>...
Attempt 1 to find and click 'Påmind mig senere' button in Shadow DOM...
Found <onboarding-modal> element.
Found 'Påmind mig senere' button in Shadow DOM.
Clicked 'Påmind mig senere' button.
Welcome pop-up dismissed.
Applying filters...
Attempt 1 to apply filters...
Filters section found.
Selecting 'København og omegn' filter...
Found 'København og omegn' checkbox.
Clicked 'København og omegn' filter.
Selecting 'Til salg' ad type filter...
Found 'Til salg' checkbox.
Clicked 'Til salg' ad type filter.
Setting price filter from 1500 onwards...
Found price 'from' input.
Set price 'from' to 1500.
Fou

Error navigating to page 2 with click: Message: element click intercepted: Element <a href="?page=2&amp;price_from=1500&amp;q=cykel" rel="next" class=" hover:no-underline focus:no-underline focusable inline-flex justify-center items-center transition-colors ease-in-out min-h-[44px] min-w-[44px] p-4 rounded-full border-0 hover:bg-clip-padding s-icon hover:bg-[--w-color-button-pill-background-hover] active:bg-[--w-color-button-pill-background-active]">...</a> is not clickable at point (827, 22). Other element would receive the click: <finn-topbar active-menu-item="NONE" sites="//www.dba.dk" locale="da" translations="{&quot;onboarding.dialog.content.first-screen.image.alttext.tori&quot;:[&quot;En rød cirkel med ordene \&quot;ToriDiili - sikker betaling, problemfri levering\&quot;.&quot;],&quot;onboarding.dialog.content.third-screen.image.alttext.tori&quot;:[&quot;En smartphone, der viser overskriften \&quot;Tilbyd ToriDiili - Toris nye fragt- og betalingsservice\&quot;. En pil peger på de

Navigated to page 2 using direct URL: https://www.dba.dk/recommerce/forsale/search?page=2&price_from=1500&q=cykel
Scraping page 2...
Applying filters...
Attempt 1 to apply filters...
Filters section found.
Selecting 'København og omegn' filter...
Found 'København og omegn' checkbox.
Clicked 'København og omegn' filter.
Selecting 'Til salg' ad type filter...
Found 'Til salg' checkbox.
Clicked 'Til salg' ad type filter.
Setting price filter from 1500 onwards...
Found price 'from' input.
Set price 'from' to 1500.
Found price 'to' input.
Cleared price 'to' field for any number onwards.
Found 'Søg' button for price filter.
Clicked 'Søg' to apply price filter.
Selecting 'Privat' seller filter...
Found 'Privat' checkbox.
Clicked 'Privat' seller filter.
Waiting for page to render after applying filters...
Parsing page source with BeautifulSoup...
Found 54 listings on page 2
Listing 1 on page 2 - Title: specialized Som ny - bedste el cykel på marked
Listing 1 on page 2 - Price: 22.500 kr.
Listi

Error navigating to page 3 with click: Message: element click intercepted: Element <a href="?dealer_segment=1&amp;location=0.200001&amp;page=2&amp;price_from=1500&amp;q=cykel&amp;trade_type=1" rel="next" class=" hover:no-underline focus:no-underline focusable inline-flex justify-center items-center transition-colors ease-in-out min-h-[44px] min-w-[44px] p-4 rounded-full border-0 hover:bg-clip-padding s-icon hover:bg-[--w-color-button-pill-background-hover] active:bg-[--w-color-button-pill-background-active]">...</a> is not clickable at point (1120, 45). Other element would receive the click: <finn-topbar active-menu-item="NONE" sites="//www.dba.dk" locale="da" translations="{&quot;onboarding.dialog.content.first-screen.image.alttext.tori&quot;:[&quot;En rød cirkel med ordene \&quot;ToriDiili - sikker betaling, problemfri levering\&quot;.&quot;],&quot;onboarding.dialog.content.third-screen.image.alttext.tori&quot;:[&quot;En smartphone, der viser overskriften \&quot;Tilbyd ToriDiili - To

Navigated to page 3 using direct URL: https://www.dba.dk/recommerce/forsale/search?dealer_segment=1&location=0.200001&page=2&price_from=1500&q=cykel&trade_type=1
Scraping page 3...
Applying filters...
Attempt 1 to apply filters...
Filters section found.
Selecting 'København og omegn' filter...
Found 'København og omegn' checkbox.
Clicked 'København og omegn' filter.
Selecting 'Til salg' ad type filter...
Found 'Til salg' checkbox.
Clicked 'Til salg' ad type filter.
Setting price filter from 1500 onwards...
Found price 'from' input.
Set price 'from' to 1500.
Found price 'to' input.
Cleared price 'to' field for any number onwards.
Found 'Søg' button for price filter.
Clicked 'Søg' to apply price filter.
Selecting 'Privat' seller filter...
Found 'Privat' checkbox.
Clicked 'Privat' seller filter.
Waiting for page to render after applying filters...
Parsing page source with BeautifulSoup...
Found 54 listings on page 3
Listing 1 on page 3 - Title: Focus cykel
Listing 1 on page 3 - Price: 3.7

...

Navigated to page 11 using direct URL: https://www.dba.dk/recommerce/forsale/search?dealer_segment=1&location=0.200001&page=2&price_from=1500&q=cykel&trade_type=1
Scraping page 11...
Applying filters...
Attempt 1 to apply filters...
Filters section found.
Selecting 'København og omegn' filter...
Found 'København og omegn' checkbox.
Clicked 'København og omegn' filter.
Selecting 'Til salg' ad type filter...
Found 'Til salg' checkbox.
Clicked 'Til salg' ad type filter.
Setting price filter from 1500 onwards...
Found price 'from' input.
Set price 'from' to 1500.
Found price 'to' input.
Cleared price 'to' field for any number onwards.
Found 'Søg' button for price filter.
Clicked 'Søg' to apply price filter.
Selecting 'Privat' seller filter...
Found 'Privat' checkbox.
Clicked 'Privat' seller filter.
Waiting for page to render after applying filters...
Parsing page source with BeautifulSoup...
Found 54 listings on page 11
Listing 1 on page 11 - Title: E fly el cykel
Listing 1 on page 11 - Pr

Error navigating to page 12 with click: Message: element click intercepted: Element <a href="?page=2&amp;price_from=1500&amp;q=cykel" rel="next" class=" hover:no-underline focus:no-underline focusable inline-flex justify-center items-center transition-colors ease-in-out min-h-[44px] min-w-[44px] p-4 rounded-full border-0 hover:bg-clip-padding s-icon hover:bg-[--w-color-button-pill-background-hover] active:bg-[--w-color-button-pill-background-active]">...</a> is not clickable at point (1120, 45). Other element would receive the click: <finn-topbar active-menu-item="NONE" sites="//www.dba.dk" locale="da" translations="{&quot;onboarding.dialog.content.first-screen.image.alttext.tori&quot;:[&quot;En rød cirkel med ordene \&quot;ToriDiili - sikker betaling, problemfri levering\&quot;.&quot;],&quot;onboarding.dialog.content.third-screen.image.alttext.tori&quot;:[&quot;En smartphone, der viser overskriften \&quot;Tilbyd ToriDiili - Toris nye fragt- og betalingsservice\&quot;. En pil peger på 

Navigated to page 12 using direct URL: https://www.dba.dk/recommerce/forsale/search?page=2&price_from=1500&q=cykel
Scraping page 12...
Applying filters...
Attempt 1 to apply filters...
Filters section found.
Selecting 'København og omegn' filter...
Found 'København og omegn' checkbox.
Clicked 'København og omegn' filter.
Selecting 'Til salg' ad type filter...
Found 'Til salg' checkbox.
Clicked 'Til salg' ad type filter.
Setting price filter from 1500 onwards...
Found price 'from' input.
Set price 'from' to 1500.
Found price 'to' input.
Cleared price 'to' field for any number onwards.
Found 'Søg' button for price filter.
Clicked 'Søg' to apply price filter.
Selecting 'Privat' seller filter...
Found 'Privat' checkbox.
Clicked 'Privat' seller filter.
Waiting for page to render after applying filters...
Parsing page source with BeautifulSoup...
Found 54 listings on page 12
Listing 1 on page 12 - Title: Raleigh Classic damecykel i sort. Flot stand. 7 gear. Passer til højde 160-170cm
Listing

Navigated to page 13
Scraping page 13...
Applying filters...
Attempt 1 to apply filters...
Filters section found.
Selecting 'København og omegn' filter...
Found 'København og omegn' checkbox.
Clicked 'København og omegn' filter.
Selecting 'Til salg' ad type filter...
Found 'Til salg' checkbox.
Clicked 'Til salg' ad type filter.
Setting price filter from 1500 onwards...
Found price 'from' input.
Set price 'from' to 1500.
Found price 'to' input.
Cleared price 'to' field for any number onwards.
Found 'Søg' button for price filter.
Clicked 'Søg' to apply price filter.
Selecting 'Privat' seller filter...
Found 'Privat' checkbox.
Clicked 'Privat' seller filter.
Waiting for page to render after applying filters...
Parsing page source with BeautifulSoup...
Found 54 listings on page 13
Listing 1 on page 13 - Title: Herrecykel, Mustang Touring, EL-cykel, 8 gear, velholdt.
Listing 1 on page 13 - Price: 6.000 kr.
Listing 1 on page 13 - Location: Hvidovre
Listing 1 on page 13 - Time Posted: to dage

Navigated to page 14
Scraping page 14...
Applying filters...
Attempt 1 to apply filters...
Filters section found.
Selecting 'København og omegn' filter...
Found 'København og omegn' checkbox.
Clicked 'København og omegn' filter.
Selecting 'Til salg' ad type filter...
Found 'Til salg' checkbox.
Clicked 'Til salg' ad type filter.
Setting price filter from 1500 onwards...
Found price 'from' input.
Set price 'from' to 1500.
Found price 'to' input.
Cleared price 'to' field for any number onwards.
Found 'Søg' button for price filter.
Clicked 'Søg' to apply price filter.
Selecting 'Privat' seller filter...
Found 'Privat' checkbox.
Clicked 'Privat' seller filter.
Waiting for page to render after applying filters...
Parsing page source with BeautifulSoup...
Found 54 listings on page 14
Listing 1 on page 14 - Title: Meget velholdt Zaskar Racing letvægts racer med 16 gear. (Stelstr 56).
Listing 1 on page 14 - Price: 1.750 kr.
Listing 1 on page 14 - Location: Charlottenlund
Listing 1 on page 14 - 